In [0]:
from IPython.display import clear_output

In [0]:
!pip3 install pycodestyle flake8 pycodestyle_magic
!pip3 install pymorphy2[fast]
!pip3 install python-rake
clear_output()

In [0]:
%load_ext pycodestyle_magic

In [0]:
import re
import nltk
import RAKE
import pandas as pd
import numpy as np
from pprint import pprint
from bs4 import BeautifulSoup
from string import punctuation
from pymorphy2 import MorphAnalyzer
from urllib.request import urlopen, Request
from gensim.summarization import keywords as kw
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
punctuation = punctuation + '«»—–’'

#**Data preprocessing**
Тексты взяты с интернет-ресурса [dtf.ru](https://dtf.ru), посвящённого фильмам и компьютерным играм, разделов "Игры" и "Кино и сериалы". Тематика текстов усложняет нам задачу, т. к., в текстах, посвященных кино и играм, в среднем больше именованных сущностей, чем в текстах других категорий. <br>
Каждая статья включала в себя тэги по ключевым словам. Отбирались только статьи минимум с тремя тэгами. Не рассматривались: <br>
*   Тэги, состоящие из нескольких "склеенных" слов (например, #хищныептицы, #stardewvalley, #корейскоекино)
*   Тэги, не являющиеся ключевыми словами, а указывающие на категорию текста (например, #новости, #истории, #расследования), локацию, в которой происходит действие текста (например, #азия) или какое-либо ещё свойство текста
*   Тэги, не встречающиеся в тексте статьи <br>

Источники: <br>
1.   https://dtf.ru/cinema/83621-pewdiepie-o-reakcii-smi-na-film-dzhoker-smi-plodyat-prestupnikov-svoim-vnimaniem
2.   https://dtf.ru/games/84132-steam-autumn-sale-delimsya-interesnymi-igrami-kotorye-stoit-kupit
3.   https://dtf.ru/cinema/82928-netflix-zakazal-prodolzhenie-seriala-vikingi-ot-history-ego-sobytiya-razvernutsya-spustya-100-let 
4.   https://dtf.ru/games/83548-half-life-alyx-eto-ne-tolko-prikvel-k-half-life-2-informaciya-i-detali-o-novoy-half-life
5.   https://dtf.ru/cinema/82054-trup-nevesty-poluchit-novuyu-ekranizaciyu




In [0]:
stopwords = nltk.corpus.stopwords.words('russian')

In [0]:
morph = MorphAnalyzer()

In [0]:
def download_text(url):
    """
    Download text and extract keywords.

    Parameters:
    url : str, an url to the text

    Returns: Dict()
    Keys and values:
    'text' : str, original text
    'lemmas' : str, lemmatized text
    'keywords' : list(str), text hashtags
    """
    page = urlopen(Request(url=url)).read()
    page_parsed = BeautifulSoup(page, 'html.parser')
    title = page_parsed.title.string
    title = title.replace(' — Кино и сериалы на DTF', '')
    title = title.replace(' — Игры на DTF', '')
    tags = page_parsed.find_all('div', class_='layout--a')
    text = ' '.join([' '.join(
        [p.get_text() for p in tag.find_all('p')]) for tag in tags
        if not 'comments__content' in tag['class'] and tag.find('p')])
    text = title + ' ' + text
    keywords = [word[1:] for word in list(
        re.findall(r'#[a-zA-Zа-яА-ЯёЁ\d]+', text))]
    text = re.sub(r'#[a-zA-Zа-яА-ЯёЁ\d]+', '', text).strip()
    lemmas = ' '.join([morph.parse(word)[0].normal_form
                       for word in nltk.word_tokenize(text)
                       if word not in stopwords
                       and word not in punctuation
                       and word not in ['``', "''"]])
    return {'text': text, 'lemmas': lemmas, 'keywords': keywords}

In [0]:
corpus = []

In [0]:
text_links = [
              'https://dtf.ru/cinema/83621-pewdiepie-o-reakcii-smi-na-film-dzhoker-smi-plodyat-prestupnikov-svoim-vnimaniem',
              'https://dtf.ru/games/84132-steam-autumn-sale-delimsya-interesnymi-igrami-kotorye-stoit-kupit',
              'https://dtf.ru/cinema/82928-netflix-zakazal-prodolzhenie-seriala-vikingi-ot-history-ego-sobytiya-razvernutsya-spustya-100-let',
              'https://dtf.ru/games/83548-half-life-alyx-eto-ne-tolko-prikvel-k-half-life-2-informaciya-i-detali-o-novoy-half-life',
              'https://dtf.ru/cinema/82054-trup-nevesty-poluchit-novuyu-ekranizaciyu'
]

In [0]:
for link in text_links:
    corpus.append(download_text(link))

In [13]:
corpus[0]

{'keywords': ['youtube', 'pewdipie', 'джокер'],
 'lemmas': 'pewdiepie реакция сми фильм джокер сми плодить преступник свой внимание в последний ролик рассуждать насколько оправдать такой пристальный внимание фильм оно навредить общество pewdiepie называть реакция фильм сторона сми странный упоминать некоторый заголовок рецензия фильм джокер содержимый выражение опасный сообщение мужской ярость культура инцеловый белый ярость такой образ намеренно готовиться тот произойти что-то плохой но сюрприз фильм выйти плохой произойти кроме кучка ребёнок танцующий лестница по слово сми понять 7 год назад премьер фильм тёмный рыцарь возрождение легенда случиться один трагедия 24-летний джеймс холмс расстрелять 12 человек в сеть распространить версия убийца вдохновляться джокер просить называть однако это просто хотеть убить человек поэтому прийти кинотеатр самый популярный фильм он мотивация просто почувствовать история джокер являться выдумка сми который слишком ложиться реальность упоминать обви

In [14]:
pprint(corpus[0]['text'])

('PewDiePie о реакции СМИ на фильм "Джокер": СМИ плодят преступников своим '
 'вниманием В последнем ролике он рассуждает о том, насколько оправдано такое '
 'пристальное внимание к фильму и чем оно может навредить обществу. PewDiePie '
 'называет реакцию на фильм со стороны СМИ "странной", упоминая некоторые '
 'заголовки рецензий на фильм Джокер, содержащие выражения "опасное '
 'сообщение", "мужская ярость", "культура инцелов" и "белая ярость". Таким '
 'образом, они как будто намеренно готовились к тому, что произойдёт что-то '
 'плохое. Но, "сюрприз: фильм вышел и ничего плохого не произошло, кроме кучки '
 'детей, танцующих на лестнице".  По его словам, СМИ можно понять, ведь 7 лет '
 'назад на премьере фильма "Тёмный рыцарь: возрождение легенды" уже случилась '
 'одна трагедия: 24-летний Джеймс Холмс расстрелял 12 человек. В сети '
 'распространена версия о том, что убийца вдохновлялся Джокером и сам просил '
 'его так называть, однако это не так - он просто хотел убить как можн

In [15]:
pprint(corpus[0]['lemmas'])

('pewdiepie реакция сми фильм джокер сми плодить преступник свой внимание в '
 'последний ролик рассуждать насколько оправдать такой пристальный внимание '
 'фильм оно навредить общество pewdiepie называть реакция фильм сторона сми '
 'странный упоминать некоторый заголовок рецензия фильм джокер содержимый '
 'выражение опасный сообщение мужской ярость культура инцеловый белый ярость '
 'такой образ намеренно готовиться тот произойти что-то плохой но сюрприз '
 'фильм выйти плохой произойти кроме кучка ребёнок танцующий лестница по слово '
 'сми понять 7 год назад премьер фильм тёмный рыцарь возрождение легенда '
 'случиться один трагедия 24-летний джеймс холмс расстрелять 12 человек в сеть '
 'распространить версия убийца вдохновляться джокер просить называть однако '
 'это просто хотеть убить человек поэтому прийти кинотеатр самый популярный '
 'фильм он мотивация просто почувствовать история джокер являться выдумка сми '
 'который слишком ложиться реальность упоминать обвинение стор

In [16]:
corpus[1]

{'keywords': ['sale', 'steam', 'распродажа', 'стим', 'скидки', 'подборка'],
 'lemmas': 'steam autumn sale делиться интересный игра который стоить купить привет хотеть организовать местечко пользователь дтф мочь рассказать тот игра который ваш мнение стоить приобрести распродажа и здорово топика смочь найти известный весь ааа-игра который список желаемый скрытый жемчужина стим который последний год сложный найти для начало предложить несколько жемчужина как видно описание это портал+зельд смешанный метроидвания абсолютно прелестный сюжетный песочница буквально песочница разработать один человек играть роль рядовой солдат который предстоять раскрыть божий замысел цена 228 pуб 40 пиксельный рогалик элемент рпг напоминать смесь диабла дарк соулс у игра приятный визуальный стиль увлекательный сюжет повествовать судьба семья бергсон цена 348 pуб 25 интересный изометрический экшен-рпг элемент стелс кибер-фентези сеттинг создатель ведьмак 3 который выход игра основать студия покинуть cdpr цена

In [17]:
pprint(corpus[1]['text'])

('Steam Autumn Sale. Делимся интересными играми, которые стоит купить Привет, '
 'хочу организовать местечко, где пользователи ДТФ могли бы рассказать о тех '
 'играх, которые по вашему мнению стоит приобрести на этой распродаже. И будет '
 'здорово, если в этом топике мы сможем найти не только известные всем '
 'ААА-игры которые уже есть, (или есть в списке желаемого) но и "скрытые '
 'жемчужины" Стима, которые в последние годы все сложнее найти. Для начала я '
 'предложу несколько "жемчужин". Как видно из описания - это Портал+Зельда '
 'смешанные с метроидванией. Абсолютно прелестная сюжетная песочница(буквально '
 'песочница) разработанная одним человеком, где вы играете роль рядового '
 'солдата которому предстоит раскрыть "божий замысел". Цена: 228 pуб.(40%) '
 'Пиксельный рогалик с элементами РПГ напоминающий смесь Диаблы и Дарк Соулс. '
 'У игры приятный визуальный стиль и увлекательный сюжет, повествующий о '
 'судьбе семьи Бергсонов. Цена: 348 pуб.(25%) Интересная изометричес

In [18]:
pprint(corpus[1]['lemmas'])

('steam autumn sale делиться интересный игра который стоить купить привет '
 'хотеть организовать местечко пользователь дтф мочь рассказать тот игра '
 'который ваш мнение стоить приобрести распродажа и здорово топика смочь найти '
 'известный весь ааа-игра который список желаемый скрытый жемчужина стим '
 'который последний год сложный найти для начало предложить несколько '
 'жемчужина как видно описание это портал+зельд смешанный метроидвания '
 'абсолютно прелестный сюжетный песочница буквально песочница разработать один '
 'человек играть роль рядовой солдат который предстоять раскрыть божий замысел '
 'цена 228 pуб 40 пиксельный рогалик элемент рпг напоминать смесь диабла дарк '
 'соулс у игра приятный визуальный стиль увлекательный сюжет повествовать '
 'судьба семья бергсон цена 348 pуб 25 интересный изометрический экшен-рпг '
 'элемент стелс кибер-фентези сеттинг создатель ведьмак 3 который выход игра '
 'основать студия покинуть cdpr цена 209 pуб 70 совершенно великолепный '


In [19]:
corpus[2]

{'keywords': ['викинги', 'netflix', 'history', 'новости'],
 'lemmas': 'netflix заказать продолжение сериал викинг history событие развернуться спустя 100 год шесть финальный сезон сериал викинг history стартовать 4 декабрь однако он франшиза закончиться как сообщать variety переехать netflix выйти шоу-сиквел название викинг вальгалла событие вальгалла развернуться спустя 100 год финал оригинальный сериал сюжет рассказать приключение такой викинг лейф эриксон фрейдис харальда iii суровый вильгельм завоеватель исполнительный продюсер новое сериал выступить создатель викинг майкл хёрстый шоураннер стать джести стюарт сценарист крепкий орешек беглец продюсер оригинал морган о салливан также вернуться сериал викинг history начать выходить март 2013 год сей пора он появиться 69 эпизод заключительный шесть сезон состоять 20 серия предыдущий',
 'text': 'Netflix заказал продолжение сериала «Викинги» от History — его события развернутся спустя 100 лет Шестой и финальный сезон сериала «Викинги» о

In [20]:
pprint(corpus[2]['text'])

('Netflix заказал продолжение сериала «Викинги» от History — его события '
 'развернутся спустя 100 лет Шестой и финальный сезон сериала «Викинги» от '
 'History стартует 4 декабря, однако на нём франшиза не закончится. Как '
 'сообщает Variety, она переедет на Netflix, где выйдет шоу-сиквел под '
 'названием «Викинги: Вальгалла». События «Вальгаллы» развернутся спустя 100 '
 'лет после финала оригинального сериала — сюжет расскажет о приключениях '
 'таких викингов, как Лейф Эриксон, Фрейдис, Харальд III Суровый и Вильгельм '
 'Завоеватель. Исполнительным продюсером нового сериала выступит создатель '
 '«Викингов» Майкл Хёрст, а шоураннером стал Джеб Стюарт, сценарист «Крепкого '
 'орешка» и «Беглеца». Продюсер оригинала Морган О’Салливан также вернётся.  '
 'Сериал «Викинги» от History начал выходить в марте 2013 года и до сих пор в '
 'нём появилось 69 эпизодов. Заключительный шестой сезон будет состоять из 20 '
 'серий, как и предыдущие два.')


In [21]:
pprint(corpus[2]['lemmas'])

('netflix заказать продолжение сериал викинг history событие развернуться '
 'спустя 100 год шесть финальный сезон сериал викинг history стартовать 4 '
 'декабрь однако он франшиза закончиться как сообщать variety переехать '
 'netflix выйти шоу-сиквел название викинг вальгалла событие вальгалла '
 'развернуться спустя 100 год финал оригинальный сериал сюжет рассказать '
 'приключение такой викинг лейф эриксон фрейдис харальда iii суровый вильгельм '
 'завоеватель исполнительный продюсер новое сериал выступить создатель викинг '
 'майкл хёрстый шоураннер стать джести стюарт сценарист крепкий орешек беглец '
 'продюсер оригинал морган о салливан также вернуться сериал викинг history '
 'начать выходить март 2013 год сей пора он появиться 69 эпизод заключительный '
 'шесть сезон состоять 20 серия предыдущий')


In [22]:
corpus[3]

{'keywords': ['halflife', 'halflife3', 'Alyx', 'Valve', 'VR', 'valveindex'],
 'lemmas': 'half-life alyx это приквёлый half-life 2 информация деталь новый half-life о новый half-life написать мало дтф однако сила различный причина большой часть информация проект ещё никто писать я решить исправить ситуация ниже постараться донести известный текущий момент информация half-life alyx один плейтест игра проводиться ещё 4.5 год назад над игра работать самый большой команда история valve треть разработчик работать half-life 2 half-life марк лейдло сценарист оригинальный дилогия half-life portal который покинуть valve помогать писатель работа half-life alyx half-life alyx вырасти техно-vr демки destinations именно уникальный возможность vr лечь основа игра в февраль 2017 гейба ньюэла сообщать разработка находиться 3 три полноценный vr-проект в игра помимо грави-перчатка присутствовать мультиинструмент алекс который использовать half-life 2 в иной вид игра присутствовать реалистичный инвентарь 

In [23]:
pprint(corpus[3]['text'])

('Half-Life: Alyx – это не только приквел к Half-Life 2. Информация и детали о '
 'новой Half-Life О новой Half-Life уже написано не мало на ДТФ, однако в силу '
 'различных причин большую часть информации об этом проекте ещё никто не '
 'писал. Я решил исправить ситуацию. Ниже постараюсь донести известную на '
 'текущий момент информацию о Half-Life: Alyx. Первые плейтесты игры '
 'проводились ещё 4.5 года назад. Над игрой работает самая большая команда в '
 'истории Valve, треть из разработчиков работала над Half-Life 2 и Half-Life. '
 'Марк Лейдло (сценарист оригинальной дилогии Half-Life и Portal), который '
 'покинул Valve, помогает писателям с работой над Half-Life: Alyx. Half-Life: '
 'Alyx выросла из техно-VR демки Destinations, именно уникальные возможности '
 'VR легли в основу игры. В феврале 2017 Гейб Ньюэл сообщал, что в разработке '
 'находятся 3(ТРИ!) полноценных VR-проекта. В игре помимо грави-перчаток '
 'присутствует «мультиинструмент» Алекс который она использовала в

In [24]:
pprint(corpus[3]['lemmas'])

('half-life alyx это приквёлый half-life 2 информация деталь новый half-life о '
 'новый half-life написать мало дтф однако сила различный причина большой '
 'часть информация проект ещё никто писать я решить исправить ситуация ниже '
 'постараться донести известный текущий момент информация half-life alyx один '
 'плейтест игра проводиться ещё 4.5 год назад над игра работать самый большой '
 'команда история valve треть разработчик работать half-life 2 half-life марк '
 'лейдло сценарист оригинальный дилогия half-life portal который покинуть '
 'valve помогать писатель работа half-life alyx half-life alyx вырасти '
 'техно-vr демки destinations именно уникальный возможность vr лечь основа '
 'игра в февраль 2017 гейба ньюэла сообщать разработка находиться 3 три '
 'полноценный vr-проект в игра помимо грави-перчатка присутствовать '
 'мультиинструмент алекс который использовать half-life 2 в иной вид игра '
 'присутствовать реалистичный инвентарь расположить самый персонаж кроме игра '

In [25]:
corpus[4]

{'keywords': ['мультфильм', 'экранизация', 'ремейк'],
 'lemmas': 'труп невеста получить новый экранизация как минимум ещё повод ждать 2021 год вчера стать известно warner brothers снова собираться экранизировать знаменитый картина тим бёртон труп невеста 2021 год но о весь порядок для начало хотеть предупредить текст возможный спойлер если вы смотреть мультфильм предлагать сначала ознакомиться затем дать статья остальной приятный чтение за последний пара год создать довольно современный экранизация тот мультфильм который многие смотреть ещё кассета золушка король левый тарзан алладин др некоторый удачный некоторый менее подобный ремейка быть создаваться смотреть возможный недовольство зритель теперь стать известно новый экранизация который выйти 2021 год обещать warner brothers труп невеста кукольный мультипликация история любовь который также раскрывать подробность неудавшийся жизнь девушка эмиль который убитый свадьба собственный жених это добрый одновременно грустный история который

In [26]:
pprint(corpus[4]['text'])

('«Труп невесты» получит новую экранизацию Как минимум ещё один повод ждать '
 '2021 год.  Вчера стало известно о том, что Warner Brothers снова собираются '
 'экранизировать знаменитую картину Тима Бёртона «Труп невесты» к 2021 году. '
 'Но обо всём по порядку.  Для начала хочу предупредить о том, что в тексте '
 'возможны спойлеры!  Если Вы не смотрели мультфильм, то предлагаю сначала '
 'ознакомиться с ним, а затем с данной статьёй.  Остальным же - приятного '
 'чтения!  За последние пару лет было создано довольно много современных '
 'экранизаций тех мультфильмов, которые многие смотрели ещё на кассетах.  '
 '«Золушка», «Король Лев», «Тарзан» «Алладин» и др. - некоторые более удачные, '
 'некоторые менее, но подобные ремейки будут создаваться, не смотря на '
 'возможное недовольство зрителя.  Теперь стало известно о новой экранизации, '
 'которая выйдет до 2021 года, как обещают Warner Brothers.  «Труп невесты» - '
 'кукольная мультипликация о истории любви, которая также раскрывае

In [27]:
pprint(corpus[4]['lemmas'])

('труп невеста получить новый экранизация как минимум ещё повод ждать 2021 год '
 'вчера стать известно warner brothers снова собираться экранизировать '
 'знаменитый картина тим бёртон труп невеста 2021 год но о весь порядок для '
 'начало хотеть предупредить текст возможный спойлер если вы смотреть '
 'мультфильм предлагать сначала ознакомиться затем дать статья остальной '
 'приятный чтение за последний пара год создать довольно современный '
 'экранизация тот мультфильм который многие смотреть ещё кассета золушка '
 'король левый тарзан алладин др некоторый удачный некоторый менее подобный '
 'ремейка быть создаваться смотреть возможный недовольство зритель теперь '
 'стать известно новый экранизация который выйти 2021 год обещать warner '
 'brothers труп невеста кукольный мультипликация история любовь который также '
 'раскрывать подробность неудавшийся жизнь девушка эмиль который убитый '
 'свадьба собственный жених это добрый одновременно грустный история который '
 'действитель

Посчитаем общее количество токенов в лемматизированном корпусе.

In [28]:
sum([len(text['lemmas'].split()) for text in corpus])

1060

Тэги, которые необходимо удалить:
1.   Из статьи про Steam Autumn Sale – "скидки" (не встречается в тексте), "подборка" (не является ключевым словом).
2.   Из статьи про сериал "Викинги" – "новости" (не является ключевым словом).
3.   Из статьи про Half-Life: Alyx – "halflife3" (не встречется в тексте), "valveindex" (состоит из нескольких слов). Кроме того, тэг "halflife" необходимо заменить на "half-life", чтобы написание соответствовало использующемуся в статье. <br>

Все тэги я привела к нижнему регистру.

In [0]:
corpus[1]['keywords'] = ['sale', 'steam', 'распродажа', 'стим']
corpus[2]['keywords'] = ['викинги', 'netflix', 'history']
corpus[3]['keywords'] = ['half-life', 'alyx', 'valve', 'vr']

#**Manually selecting keywords**
Для первого текста я выделяю ключевые слова 'pewdiepie', 'сми' и 'джокер'. Отличия от имеющейся разметки только в одном слове.

In [0]:
corpus[0]['keywords_manual'] = ['pewdipie', 'сми', 'джокер']

Для второго текста я согласна с имеющейся разметкой (ключевые слова – 'sale', 'steam', 'распродажа', 'стим').

In [0]:
corpus[1]['keywords_manual'] = corpus[1]['keywords']

Для третьего текста я согласна с имеющейся разметкой(ключевые слова – 'викинги', 'netflix', 'history').

In [0]:
corpus[2]['keywords_manual'] = corpus[2]['keywords']

Для четвёртого текста я согласна с имеющейся разметкой(ключевые слова – 'half-life', 'alyx', 'valve', 'vr').

In [0]:
corpus[3]['keywords_manual'] = corpus[3]['keywords']

Для пятого текста я согласна с имеющейся разметкой(ключевые слова – 'мультфильм', 'экранизация', 'ремейк').

In [0]:
corpus[4]['keywords_manual'] = corpus[4]['keywords']

Имеющаяся разметка отличается от моей ровно на одно слово – в первом тексте изначально был тэг 'youtube', который я заменила на 'сми'. Слово 'youtube' встречается в тексте 1 раз и очень слабо связано с содержанием, слово 'сми' встречается 7 раз и напрямую связано с содержанием и тематикой текста (текст про мнение PewDiePue о реакции СМИ на фильм "Джокер") – соответственно, логично выделять как ключевое слово именно 'сми'. Поэтому значения метрик я считала относительно своей разметки, а не относительно имеющейся.

#**Keyword extraction**

In [0]:
def print_metrics(kw_true, kw_pred):
    """
    Print precision, recall and F1-score.
    """
    precision = 0
    if len(kw_pred) > 0:
        precision = len(set(kw_pred) & set(kw_true)) / len(kw_pred)
    recall = len(set(kw_pred) & set(kw_true)) / len(kw_true)
    f1_score = 0
    if precision != 0 and recall != 0:
        f1_score = 2 * precision * recall / (precision + recall)
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1-score: {f1_score}\n')

##RAKE
На текстах без лемматизации:

In [0]:
rake = RAKE.Rake(stopwords)

In [47]:
for text in corpus:
    predictions = [pred[0] for pred in rake.run(text['text'],
                                                maxWords=3, minFrequency=2)]
    print(f"original keywords: {text['keywords']}")
    print(f"manual keywords: {text['keywords_manual']}")
    print(f"predicted keywords: {predictions}")
    print_metrics(text['keywords_manual'], predictions)

original keywords: ['youtube', 'pewdipie', 'джокер']
manual keywords: ['pewdipie', 'сми', 'джокер']
predicted keywords: ['таким образом', 'фильм', 'плохого', 'словам', 'говорит', 'например']
Precision: 0.0
Recall: 0.0
F1-score: 0

original keywords: ['sale', 'steam', 'распродажа', 'стим']
manual keywords: ['sale', 'steam', 'распродажа', 'стим']
predicted keywords: ['которые', 'цена', '25%']
Precision: 0.0
Recall: 0.0
F1-score: 0

original keywords: ['викинги', 'netflix', 'history']
manual keywords: ['викинги', 'netflix', 'history']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0

original keywords: ['half-life', 'alyx', 'valve', 'vr']
manual keywords: ['half-life', 'alyx', 'valve', 'vr']
predicted keywords: ['новой half-life', 'half-life', 'half-life 2', 'ещё', 'это', 'alyx', 'кроме']
Precision: 0.2857142857142857
Recall: 0.5
F1-score: 0.36363636363636365

original keywords: ['мультфильм', 'экранизация', 'ремейк']
manual keywords: ['мультфильм', 'экранизация', 'ремейк']
pre

Очень плохая модель. Что-то похожее на ключевые слова выделилось только в четвёртом тексте, и то только благодаря тому, что слова half-life и alyx 1) неизменяемые 2) очень часто встречаются в четвёртом тексте. Значения метрик все нули во всех текстах, кроме четвёртого. Заметим, что большинство предсказаний алгоритма – просто очень частотные словоформы.<br>
<br>
Запустим RAKE на текстах с лемматизацией.

In [49]:
for text in corpus:
    predictions = [pred[0] for pred in rake.run(text['lemmas'],
                                                maxWords=3, minFrequency=2)]
    print(f"original keywords: {text['keywords']}")
    print(f"manual keywords: {text['keywords_manual']}")
    print(f"predicted keywords: {predictions}")
    print_metrics(text['keywords_manual'], predictions)

original keywords: ['youtube', 'pewdipie', 'джокер']
manual keywords: ['pewdipie', 'сми', 'джокер']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0

original keywords: ['sale', 'steam', 'распродажа', 'стим']
manual keywords: ['sale', 'steam', 'распродажа', 'стим']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0

original keywords: ['викинги', 'netflix', 'history']
manual keywords: ['викинги', 'netflix', 'history']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0

original keywords: ['half-life', 'alyx', 'valve', 'vr']
manual keywords: ['half-life', 'alyx', 'valve', 'vr']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0

original keywords: ['мультфильм', 'экранизация', 'ремейк']
manual keywords: ['мультфильм', 'экранизация', 'ремейк']
predicted keywords: []
Precision: 0
Recall: 0.0
F1-score: 0



Внезапно на лемматизированных текстах RAKE не предсказывает вообще ничего. Причина – скорее всего, небольшой объём как самих текстов, так и списков ключевых слов, не позволяющий сделать какие-то выводы из совместной встречаемости слов.
##TextRank with gensim

In [54]:
for text in corpus:
    predictions = kw(text['lemmas']).split('\n')
    print(f"original keywords: {text['keywords']}")
    print(f"manual keywords: {text['keywords_manual']}")
    print(f"predicted keywords: {predictions}")
    print_metrics(text['keywords_manual'], predictions)

original keywords: ['youtube', 'pewdipie', 'джокер']
manual keywords: ['pewdipie', 'сми', 'джокер']
predicted keywords: ['реакция сми фильм джокер', 'это просто хотеть', 'произоити', 'говорить', 'сторона', 'общество', 'свои внимание', 'ненависть', 'называть', 'человек', 'преступник', 'такои', 'проблема', 'которыи', 'рассуждать', 'например', 'золотои павильон', 'число', 'ярость', 'упоминать', 'история', 'летнии', 'трагедия', 'поэтому', 'случиться', 'плохои', 'давать', 'воина', 'оно', 'возможныи', 'аниме', 'решать', 'опасныи', 'распространить']
Precision: 0.0
Recall: 0.0
F1-score: 0

original keywords: ['sale', 'steam', 'распродажа', 'стим']
manual keywords: ['sale', 'steam', 'распродажа', 'стим']
predicted keywords: ['которыи стоить', 'интересныи игра', 'элемент рпг', 'цена', 'это', 'наити', 'жемчужина', 'pуб', 'год', 'сюжетныи песочница', 'диабла дарк', 'топика', 'autumn sale', 'прелестныи', 'абсолютно']
Precision: 0.0
Recall: 0.0
F1-score: 0

original keywords: ['викинги', 'netflix', 

Уже лучше! Во-первых, большинство предсказаний алгоритма – именные группы (а именно они часто являются ключевыми словами). Во-вторых, большинство предсказаний как-то связаны по смыслу с содержанием и тематикой текста. На фоне RAKE, который предсказывал совершенно рандомные по смыслу слова любых частей речи, это уже огромное улучшение. :D <br>
Но помимо ключевых слов и просто слов, связанных по смыслу с текстом, алгоритм даёт очень много ошибочных предсказаний (возможно, из-за того, что тексты слишком короткие). Из-за большого процента ошибочных предсказаний и малого объёма списков ключевых слов значения метрик получаются маленькими. Расширение списков ключевых слов (так, чтобы original_keywords и predicted_keywords были примерно одинаковой длины), скорее всего, улучшило бы показатели метрик.
<br>
##TF-IDF
Преобразуем корпус в матрицу TF-IDF и ключевыми будем считать те слова, значения TF-IDF для которых самые высокие. Так как в исходных списках ключевых слов у нас лежат по 3-4 слова, возьмём верхние 5 значений.

In [0]:
vectorizer = TfidfVectorizer()

In [0]:
tf_idf = vectorizer.fit_transform(
    [text['lemmas'] for text in corpus]).T.toarray()

In [0]:
tf_idf_table = pd.DataFrame(data=tf_idf, index=vectorizer.get_feature_names())

In [63]:
for i in range(5):
    predictions = list(tf_idf_table[i].nlargest(5).index)
    print(f"original keywords: {corpus[i]['keywords']}")
    print(f"manual keywords: {corpus[i]['keywords_manual']}")
    print(f"predicted keywords: {predictions}")
    print_metrics(corpus[i]['keywords_manual'], predictions)

original keywords: ['youtube', 'pewdipie', 'джокер']
manual keywords: ['pewdipie', 'сми', 'джокер']
predicted keywords: ['сми', 'фильм', 'внимание', 'джокер', 'то']
Precision: 0.4
Recall: 0.6666666666666666
F1-score: 0.5

original keywords: ['sale', 'steam', 'распродажа', 'стим']
manual keywords: ['sale', 'steam', 'распродажа', 'стим']
predicted keywords: ['pуб', 'рпг', 'цена', 'который', 'игра']
Precision: 0.0
Recall: 0.0
F1-score: 0

original keywords: ['викинги', 'netflix', 'history']
manual keywords: ['викинги', 'netflix', 'history']
predicted keywords: ['викинг', 'сериал', 'history', '100', 'netflix']
Precision: 0.4
Recall: 0.6666666666666666
F1-score: 0.5

original keywords: ['half-life', 'alyx', 'valve', 'vr']
manual keywords: ['half-life', 'alyx', 'valve', 'vr']
predicted keywords: ['half', 'life', 'alyx', 'valve', 'игра']
Precision: 0.4
Recall: 0.5
F1-score: 0.4444444444444445

original keywords: ['мультфильм', 'экранизация', 'ремейк']
manual keywords: ['мультфильм', 'экраниза

TF-IDF уже даёт какие-то нормальные предсказания и значения метрик. Это логично, так как ключевые слова, как правило, часто встречаются в своём тексте (т. е., имеют высокий TF). Кроме того, довольно часто ключевые слова текста – имена собственные, а имена собственные обычно имеют высокий IDF (встречаются в малом количестве документов). <br>
Тем не менее, для второго текста наша модель не сделала ни одного правильного предсказания. Это, видимо, связано с тем, что ключевые слова в этот текст входят всего лишь по одному разу. Поэтому модель вместо них посчитала ключевыми бессмысленные, но частотные слова – 'pуб', 'рпг', 'цена', 'который', 'игра'.